# Music Recommendation System Project

**Still very much a WIP! It works, but the output is really not very nice!**

In this project, I built a music recommendation system using machine learning techniques. The goal is to create a model that can recommend music based on similarity with another song the user inputs.

We have a large dataset in CSV format containing information about music tracks. This dataset includes features such as artists, album name, danceability, energy, and genre, among others. I chose the features I wanted to use and also ended up settling on a KNN algorithm for this project.

In [203]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Set the style for data visualization
sns.set(style="whitegrid")

# Load the dataset (replace 'your_dataset.csv' with the actual file path)
data = pd.read_csv("dataset.csv")

data.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [204]:
missing_values = data.isnull().sum()
print("Missing Values:")
missing_values

Missing Values:


Unnamed: 0          0
track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

In [205]:
data.dropna(subset=['artists', 'album_name', 'track_name'], inplace=True)
data.drop_duplicates(subset=['track_id'], inplace=True)

data.reset_index(drop=True, inplace=True)

missing_values = data.isnull().sum()

print("Missing Values:")
missing_values

Missing Values:


Unnamed: 0          0
track_id            0
artists             0
album_name          0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

In [206]:
data.describe()

,Unnamed: 0,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,89740.000000,89740.000000,8.974000e+04,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000
mean,53479.005739,33.198808,2.291444e+05,0.562166,0.634458,5.283530,-8.498994,0.636973,0.087442,0.328285,0.173415,0.216971,0.469474,122.058134,3.897426
std,33410.141924,20.580640,1.129458e+05,0.176692,0.256606,3.559912,5.221518,0.480875,0.113278,0.338321,0.323849,0.194885,0.262864,30.117651,0.453437
min,0.000000,0.000000,8.586000e+03,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23766.750000,19.000000,1.730400e+05,0.450000,0.457000,2.000000,-10.322250,0.000000,0.036000,0.017100,0.000000,0.098200,0.249000,99.262750,4.000000
50%,50680.500000,33.000000,2.132955e+05,0.576000,0.676000,5.000000,-7.185000,1.000000,0.048900,0.188000,0.000058,0.132000,0.457000,122.013000,4.000000
75%,80618.500000,49.000000,2.642930e+05,0.692000,0.853000,8.000000,-5.108000,1.000000,0.085900,0.625000,0.097625,0.279000,0.682000,140.077000,4.000000
max,113999.000000,100.000000,5.237295e+06,0.985000,1.000000,11.000000,4.532000,1.000000,0.965000,0.996000,1.000000,1.000000,0.995000,243.372000,5.000000


In [207]:
data.drop(["track_id", "Unnamed: 0", "artists", "album_name", "track_name", "key", "time_signature", "mode", "track_genre"], axis=1, inplace=True)

data

,artists,popularity,duration_ms,explicit,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
0,Gen Hoshino,73,230666,False,0.676,0.4610,-6.746,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,acoustic
1,Ben Woodward,55,149610,False,0.420,0.1660,-17.235,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,acoustic
2,Ingrid Michaelson;ZAYN,57,210826,False,0.438,0.3590,-9.734,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,acoustic
3,Kina Grannis,71,201933,False,0.266,0.0596,-18.515,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,acoustic
4,Chord Overstreet,82,198853,False,0.618,0.4430,-9.681,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89735,Rainy Lullaby,21,384999,False,0.172,0.2350,-16.393,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,world-music
89736,Rainy Lullaby,22,385000,False,0.174,0.1170,-18.318,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,world-music
89737,Cesária Evora,22,271466,False,0.629,0.3290,-10.895,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,world-music
89738,Michael W. Smith,41,283893,False,0.587,0.5060,-10.889,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,world-music


Here, I chose to drop some features that seemed irrelevant. For the first two, it is because they are randomly generated IDs, which would interfere with the model, for all the names, it is because liking a song by an artist doesn't mean you will like songs by artists with similar names, and same for albums and titles, for key and time signature, it is because most people do not care what key/time signature their music is in, for mode, I am not sure what that feature symbolized, so I chose to remove them. Finally, it seems there was a problem in my data, as the every track had a genre of "acoustic".

In [208]:
correlation_matrix = data.corr()

correlations = correlation_matrix.abs().unstack().sort_values(ascending=False)

# Filter out correlations with themselves (diagonal) and keep only the top correlated pairs
top_correlations = correlations[correlations != 1.0]

print("Top Correlated Feature Pairs:")
top_correlations

# all this looks alright, no feature seem to be correlated apart from what can be expected (energy/loudness/acousticness, mainly)

ValueError: could not convert string to float: 'Gen Hoshino'

In [ ]:
scaler = StandardScaler()

data[data.columns] = scaler.fit_transform(data[data.columns])

data[data.columns]

In [ ]:
df = pd.read_csv("dataset.csv")

df.dropna(subset=['artists', 'album_name', 'track_name'], inplace=True)

df.reset_index(drop=True, inplace=True)

In [ ]:
selected_features = data[data.columns]  # Replace with your chosen features

normalized_features = (selected_features - selected_features.mean()) / selected_features.std()

k = 5  # Number of neighbors to consider
knn_model = NearestNeighbors(n_neighbors=k, metric='euclidean')  # Use Euclidean distance

knn_model.fit(normalized_features)

# Choose a target item for which you want to make recommendations
target_track_name = '21 Guns'
target_artist = 'Green Day'
target_row = df[(df['track_name'] == target_track_name) & (df['artists'] == target_artist)].index[0]

distances, indices = knn_model.kneighbors([normalized_features.iloc[target_row]])

recommended_item_indices = indices[0][1:]

print("Recommended Items:")
print(df.iloc[recommended_item_indices])